# **1.1. Matériel : vérifier que le GPU est bien sélectionné (A compléter)**

In [ ]:
!nvidia-smi

# **1.2. Installation de pytorch lightning**

In [ ]:
!pip install pytorch_lightning

# **2.1. Importation des librairies**

In [ ]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST
from torchmetrics.classification import MulticlassConfusionMatrix, Accuracy
from pytorch_lightning.loggers import CSVLogger
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import TensorBoardLogger , CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torchsummary import summary
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import numpy as np

# **3.1. Définition des paramètres d'entrainement**

In [ ]:
NUM_CLASSES = 10
BATCH_SIZE=64 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
EPOCHS=5 #@param [5, 10,20,50,100,200] {type:"raw"}
DATA_DIR="."
LOG_DIR="logs/"
ACCELERATOR= "auto" #@param ["cpu","gpu","auto"]

#**4. Téléchargement de la base de données MNIST**

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
# Init DataLoader from MNIST Dataset
train_ds = MNIST(DATA_DIR, train=True, download=True, transform=transform)
test_ds = MNIST(DATA_DIR, train=False, download=True, transform=transform)

#**4.Question 1 : Diviser données en 3 sets : train, validation & test (A COMPLETER)**

In [ ]:
targets = train_ds.targets
train_idx, valid_idx= train_test_split(np.arange(len(targets)), test_size=0.2, random_state=42, shuffle=True, stratify=targets)
train_ds_subset = Subset(train_ds, train_idx)
val_ds = Subset(train_ds, valid_idx)

# create train , test and val loaders
train_loader = DataLoader(train_ds_subset, batch_size=BATCH_SIZE)
val_loader = #???????????????????????????????????????????????????
test_loader = #??????????????????????????????????????????????????

#**5.	Afficher les 10 premières images avec désignation de leurs labels**

In [ ]:
# Get one batch of data
images, labels = next(iter(train_loader))
num_row = 2
num_col = 5
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5 * num_col, 2 * num_row))
for i in range(10):
    ax = axes[i // num_col, i % num_col]
    ax.imshow(images[i].squeeze(), cmap='gray')
    ax.set_title('Label: {}'.format(labels[i].item()))
    ax.axis('off')
plt.tight_layout()
plt.show()

#**Question 2 : Inspecter la taille des 3 jeux de données (A COMPLETER)**

In [ ]:
### ???????????????????????????????????????????????????????????????????????????
### ???????????????????????????????????????????????????????????????????????????
### ???????????????????????????????????????????????????????????????????????????
### ???????????????????????????????????????????????????????????????????????????

#**Question 3 : Inspecter le format des images (A COMPLETER)**

In [ ]:
### ???????????????????????????????????????????????????????????????????????????
### ???????????????????????????????????????????????????????????????????????????

#**6. Créer la classe MNISTModel avec ses fonctions (Code à analyser)**

In [ ]:
class MNISTModel(pl.LightningModule):
    def __init__(self, model, num_classes=NUM_CLASSES):
        super(MNISTModel, self).__init__()
        self.model = model
        self.num_classes = num_classes
        # Initialize confusion matrix metric
        self.confusion_matrix = MulticlassConfusionMatrix(num_classes=self.num_classes)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.train_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = self.train_accuracy(y_hat, y)

        # Log training loss
        self.log_dict({'train_loss':loss,"train_acc":acc}, on_step=True,prog_bar=True,logger=True, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        self.train_accuracy.reset()

    def validation_step(self, batch, batch_idx):
          x, y = batch
          y_hat = self(x)
          val_loss = F.cross_entropy(y_hat, y)
          val_acc = self.val_accuracy(y_hat, y)
          self.log_dict({'val_loss':val_loss,"val_acc":val_acc},prog_bar=True, on_step=False, on_epoch=True)
          return val_loss

    def on_validation_epoch_end(self):
        self.val_accuracy.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        test_loss = F.cross_entropy(y_hat, y)
        test_acc = self.test_accuracy(y_hat, y)
        self.log_dict({'test_loss':test_loss,"test_acc":test_acc},prog_bar=True, on_step=False, on_epoch=True)
        self.confusion_matrix.update(y_hat.argmax(dim=1), y)
        return test_loss

    def on_test_end(self):
        self.test_accuracy.reset()
        # Compute confusion matrix at the end of the test epoch
        fig_, ax_ = self.confusion_matrix.plot()
        # Reset the confusion matrix
        self.confusion_matrix.reset()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=1e-4)

#**7. Définir le premier modèle à une seule couche**

In [ ]:
# MLP Model (Model 1)
class MLP1(nn.Module):
    def __init__(self):
        super(MLP1, self).__init__()
        self.l1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        x = self.l1(x)
        return x

#**8. Lancer l'entraintement**

In [ ]:
# For MLP model
csv_logger = CSVLogger(LOG_DIR, name="MLP1", version='')

mlp1 = MLP1()
model1  = MNISTModel(mlp1)

# Initialize a trainer
trainer = pl.Trainer(
    accelerator=ACCELERATOR,
    max_epochs=EPOCHS,
    logger=csv_logger,
)

# Train the model ⚡
trainer.fit(model1, train_loader, val_loader)

# **9. Instancier et afficher le modèle**

In [ ]:
mlp1 = MLP1()
mlp1.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
summary(mlp1,(1,28,28))

# **10. Définir la fonction de visulisation des courbes d'entrainement : Acc & Loss**

In [ ]:
def plot_metrics(log_folder):
  import pandas as pd
  import matplotlib.pyplot as plt

  # Load the CSV file generated by CSVLogger
  df = pd.read_csv(f'{LOG_DIR}/{log_folder}/metrics.csv')
  train_df = df[df['train_loss_epoch'].notna()]
  val_df = df[df['val_loss'].notna()]

  # Plot training loss
  plt.plot(train_df['epoch'], train_df['train_loss_epoch'], label='Train Loss')
  plt.plot(val_df['epoch'], val_df['val_loss'], label='Validation Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Training & Validation Loss')
  plt.legend()
  plt.grid(True)
  plt.show()

  # Plot training accuracy
  plt.plot(train_df['epoch'], train_df['train_acc_epoch'], label='Train Acc')
  plt.plot(val_df['epoch'], val_df['val_acc'], label='Val Acc')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Training & Validation Accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()

# **Question 4: Évaluer modèle avec les données de test : trainer.test() (A COMPLETER)**

In [ ]:
## ???????????????????????????????????????????????????????????????????????????????????????????

# **Question 5 : Afficher les courbes d'entrainement (A COMPLETER)**

In [ ]:
## ???????????????????????????????????????????????????????????????????????????????????????????

# **Question 6 : analyser le résultat de la matrice de confusion. Que constatez-vous ?**

Votre texte d'analyse :  

# **++++++++++++++++++ PARTIE II : Modèle MLP ++++++++++++++++++**

In [ ]:
####    CODES CORRESPONDANTS AUX QUESTIONS : 7, 8, 9, 10 et 11 (page 4 de l'énoncé)

# **++++++++++++++++++ PARTIE III : Modèle CNN ++++++++++++++++++**

# **Question 12 : créer le résearu de neurones convolutionnel CNN**

In [ ]:
nb_filters = 32                         # Number of convolutional filters to use
pool_size = (2, 2)                      # Size of pooling area for max pooling
kernel_size = (3, 3)                    # Convolution kernel size

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=nb_filters, kernel_size=kernel_size, padding='valid')
        self.conv2 = ______ # A compléter avec votre code #
        self.pool = nn.MaxPool2d(___) # A compléter avec votre code #
        # Fully Connected Layers (classifier)
        # LazyLinear is useful to automatically deduce the number of input neurones
        self.fc1 = nn.LazyLinear(128)
        self.fc2 = nn.LazyLinear(NUM_CLASSES)

    def forward(self, x):
        x = F.relu(___) # A compléter avec votre code #
        x = F.relu(___) # A compléter avec votre code #
        x = self.pool(x)
        x = torch.flatten(x, 1)  # Flatten the input to feed it to the linear layers
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Usage Example
model = CNNModel()
print(model)

In [ ]:
####    CODES CORRESPONDANTS AUX QUESTIONS : 12, 13, 14 et 15 (page 5 de l'énoncé)